Empirical Application: Data Cleaning. This Jupyter notebook contains the necessary Python code to generate cleaned corn and soybean datasets for the paper "Functional Partial Least-Squares: Adaptive Estimation and Inference" by Andrii Babii, Marine Carrasco, and Idriss Tsafack.

The inputs are three DTA (STATA) datasets from Schlenker and Roberts (PNAS, 2009): yieldData.dta, weather_corn.dta, and weather_soybeans.dta. These files can be found in the dataSTATA folder here: https://www.wolfram-schlenker.info/replicationFiles/SchlenkerRoberts2009.zip

The output is two CSV files used subsequently in the data analysis: corn.csv and soybeans.csv

The code was tested on Python v3.12.7.

In [12]:
# Import libraries, set parameters, and load yield data
import pandas as pd
import numpy as np

# Parameters
monthMin = 3
monthMax = 8
yearMin = 1950
yearMax = 2020

yields = pd.read_stata("/Users/ababii/Library/CloudStorage/Dropbox/SchlenkerRoberst2009/dataSTATA/yieldData.dta")
yields = yields[['crop', 'fips', 'year', 'yield']]

yields

,crop,fips,year,yield
0,corn,1001.0,1950.0,16.753378
1,corn,1001.0,1951.0,13.106529
2,corn,1001.0,1952.0,9.073771
3,corn,1001.0,1953.0,20.368853
4,corn,1001.0,1954.0,10.516394
...,...,...,...,...
267254,soybeans,55141.0,2016.0,50.659340
267255,soybeans,55141.0,2017.0,43.383083
267256,soybeans,55141.0,2018.0,37.739132
267257,soybeans,55141.0,2019.0,40.594059


In [14]:
# Create and save corn dataset
weather = pd.read_stata("/Users/ababii/Library/CloudStorage/Dropbox/SchlenkerRoberst2009/dataSTATA/weather_corn.dta")

# Treatment (aggregation) of weather data over seasons
weather = weather[(weather['month'] >= monthMin) &
 (weather['month'] <= monthMax) & (weather['year'] >= yearMin) &
  (weather['year'] <= yearMax)]

weather = weather.groupby(['fips', 'year']).sum()
weather = weather.filter(regex='^(time|prec)')

# Treatment of yields
yields_corn = yields[(yields['crop'] == 'corn') & (yields['year'] >= yearMin) &
 (yields['year'] <= yearMax)]

# Merging of the datasets and treatment
merged_df = pd.merge(weather, yields_corn, on=['fips', 'year'])
   
df = merged_df.drop(columns = ['crop'])
df.to_csv('/Users/ababii/Library/CloudStorage/Dropbox/SchlenkerRoberst2009/corn.csv', index=False)

df

,fips,year,prec,timeMinus5C,timeMinus4C,timeMinus3C,timeMinus2C,timeMinus1C,time0C,time1C,...,time32C,time33C,time34C,time35C,time36C,time37C,time38C,time39C,time40C,yield
0,1001.0,1950.0,803.753784,184.000000,184.000000,184.000000,183.996639,183.907707,183.429425,182.874262,...,14.829615,8.688805,4.149085,1.514229,0.415744,0.021554,0.000000,0.000000,0.000000,16.753378
1,1001.0,1951.0,572.395752,184.000000,184.000000,184.000000,184.000000,183.992126,183.784084,183.145759,...,28.446340,21.752869,15.266515,9.216571,3.694546,0.858274,0.142949,0.001342,0.000000,13.106529
2,1001.0,1952.0,657.488708,184.000000,184.000000,184.000000,184.000000,183.999905,183.909682,183.422415,...,27.785276,22.260228,16.219722,10.411566,5.444372,2.614518,1.167644,0.505065,0.221014,9.073771
3,1001.0,1953.0,717.175354,184.000000,184.000000,184.000000,184.000000,184.000000,183.999688,183.953610,...,21.948122,15.804277,9.858780,4.891759,1.115819,0.045693,0.000000,0.000000,0.000000,20.368853
4,1001.0,1954.0,383.431183,184.000000,184.000000,183.961999,183.782313,183.556023,183.111219,182.420685,...,30.093509,25.041382,19.342989,13.222932,7.987585,4.092013,1.729693,0.488791,0.074055,10.516394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128180,55141.0,2016.0,659.221191,179.977567,178.412895,176.334483,173.877537,170.916298,167.335943,163.240062,...,0.123413,0.000229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,157.070068
128181,55141.0,2017.0,691.832336,175.227113,173.689580,172.287964,170.986048,169.043494,165.995360,161.142500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,138.453613
128182,55141.0,2018.0,570.245728,170.357270,166.628571,162.250769,158.255649,153.559961,149.911283,146.271942,...,1.045381,0.365986,0.010807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,155.652176
128183,55141.0,2019.0,689.800232,171.680159,170.120244,168.312657,165.801395,162.827509,159.657266,156.134329,...,0.025284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,141.007187


In [15]:
# Create and save soybean dataset
weather = pd.read_stata("/Users/ababii/Library/CloudStorage/Dropbox/SchlenkerRoberst2009/dataSTATA/weather_soybeans.dta")

# Treatment (aggregation) of weather data over seasons
weather = weather[(weather['month'] >= monthMin) &
 (weather['month'] <= monthMax) & (weather['year'] >= yearMin) &
  (weather['year'] <= yearMax)]

weather = weather.groupby(['fips', 'year']).sum()
weather = weather.filter(regex='^(time|prec)')

# Treatment of yields
yields_soybeans = yields[(yields['crop'] == 'soybeans') & (yields['year'] >= yearMin) &
 (yields['year'] <= yearMax)]

# Merging of the datasets and treatment
merged_df = pd.merge(weather, yields_soybeans, on=['fips', 'year'])

df = merged_df.drop(columns = ['crop'])
df.to_csv('/Users/ababii/Library/CloudStorage/Dropbox/SchlenkerRoberst2009/soybeans.csv', index=False)

df

,fips,year,prec,timeMinus5C,timeMinus4C,timeMinus3C,timeMinus2C,timeMinus1C,time0C,time1C,...,time32C,time33C,time34C,time35C,time36C,time37C,time38C,time39C,time40C,yield
0,1001.0,1950.0,798.420532,184.000000,184.000000,184.000000,183.996035,183.894483,183.404555,182.840477,...,15.016437,8.823438,4.276257,1.563989,0.454081,0.031160,0.000000,0.000000,0.000000,15.000000
1,1001.0,1951.0,568.071167,184.000000,184.000000,184.000000,184.000000,183.990728,183.760062,183.134122,...,28.527290,21.802343,15.334569,9.288286,3.817469,0.931639,0.139734,0.001038,0.000000,15.000000
2,1001.0,1952.0,657.922485,184.000000,184.000000,184.000000,184.000000,183.999894,183.899181,183.409572,...,27.847727,22.346342,16.355778,10.538268,5.509559,2.627310,1.167199,0.503149,0.215804,14.000000
3,1001.0,1953.0,719.372681,184.000000,184.000000,184.000000,184.000000,184.000000,183.999376,183.952294,...,22.028430,15.904642,9.980077,4.991942,1.187952,0.054713,0.000000,0.000000,0.000000,18.000000
4,1001.0,1954.0,385.620331,184.000000,184.000000,183.956488,183.782072,183.555218,183.104635,182.404987,...,30.076401,25.022231,19.315284,13.200590,7.958173,4.038384,1.676803,0.453972,0.067690,12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102700,55141.0,2016.0,658.865723,179.974421,178.403236,176.324623,173.869659,170.909271,167.332271,163.229144,...,0.121251,0.000191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.659340
102701,55141.0,2017.0,690.967285,175.226924,173.687023,172.282545,170.985305,169.038693,165.995260,161.138883,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43.383083
102702,55141.0,2018.0,570.879517,170.369897,166.638542,162.242698,158.255944,153.543191,149.902365,146.260041,...,1.043945,0.364058,0.008679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.739132
102703,55141.0,2019.0,688.582275,171.687711,170.127205,168.324635,165.816704,162.834493,159.662582,156.136583,...,0.025822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.594059
